In [2]:
import sqlite3
import streamlit as st
import pandas as pd
import plotly.express as px

In [3]:
# ========================== Database Connection ==========================
def get_connection():
    """Establish connection to the SQLite database."""
    return sqlite3.connect("../../sqllite3_db/foodbot.db")

In [4]:
def fetch_order_data():
    """Fetch order data from the database and return it as a DataFrame."""
    query = "SELECT * FROM orders;"
    conn = get_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [5]:
# ========================== Data Processing ==========================
def preprocess_data(df):
    """Process order data to extract date, month, and year."""
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['month_name'] = df['date'].dt.month_name()
    return df

In [6]:
def calculate_monthly_revenue(df):
    """Compute monthly revenue and format the date."""
    monthly_revenue = df.groupby(['year', 'month'])['total_price'].sum().reset_index()
    monthly_revenue['date'] = pd.to_datetime(monthly_revenue[['year', 'month']].assign(day=1))
    return monthly_revenue

In [7]:
# ========================== Plotting Function ==========================
def create_revenue_chart(df):
    """Generate an interactive revenue trend line chart with a dark theme."""
    fig = px.line(
        df, 
        x="date", 
        y="total_price", 
        title="📊 Monthly Revenue Trends",
        labels={"total_price": "Revenue ($)", "date": "Month"},
        markers=True,
        line_shape="spline",  # Smooth curve
        color_discrete_sequence=["#00C3FF"]  # Bright blue line
    )

    # Customize layout for Dark Mode
    fig.update_layout(
        template="plotly_dark",  # Dark theme
        title_font=dict(size=22, family="Arial", color="#FFFFFF"),  # Stylish title
        xaxis_title="Month",
        yaxis_title="Revenue ($)",
        xaxis=dict(
            showgrid=False, 
            tickangle=45, 
            tickfont=dict(size=12, color="#FFFFFF")
        ),
        yaxis=dict(
            showgrid=True, 
            gridcolor="rgba(255,255,255,0.2)", 
            tickfont=dict(size=12, color="#FFFFFF")
        ),
        hovermode="x unified",  # Show hover info for all points on same x-axis
        plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent background
        paper_bgcolor="#121212",  # Dark background
        margin=dict(l=60, r=30, t=60, b=60),
    )

    # Add hover effects and animation
    fig.update_traces(
        hoverinfo="text+name",
        line=dict(width=3),  # Thicker line
        marker=dict(size=8, symbol="circle", line=dict(width=2, color="white")),
        mode="lines+markers"
    )

    return fig

In [8]:
# ========================== Main Execution ==========================
def main():
    """Main function to execute the workflow."""
    # Fetch data
    raw_data = fetch_order_data()
    
    # Process data
    processed_data = preprocess_data(raw_data)
    
    # Compute monthly revenue
    monthly_revenue = calculate_monthly_revenue(processed_data)
    
    # Generate the chart
    revenue_chart = create_revenue_chart(monthly_revenue)
    
    # Show the figure
    revenue_chart.show()

# Run the script
if __name__ == "__main__":
    main()


In [11]:
data = fetch_order_data()
data = preprocess_data(data)

In [12]:
from collections import Counter
import json
all_items = Counter()

In [18]:
for order in data['items']:
    order_dict = json.loads(order.replace("'", "\""))
    all_items.update(order_dict)

In [23]:
product_counts = pd.DataFrame(all_items.items(), columns=["Product", "Total Orders"]).sort_values(by="Total Orders", ascending=False)

In [ ]:
data["time"] = pd.to_datetime(data["time"], format="%I:%M:%S %p")
data['hour'] = data['time'].dt.hour
hourly_demand = data.groupby("hour")["id"].count().reset_index()
hourly_demand.columns = ["Hour", "Total Orders"]
hourly_demand

,Hour,Total Orders
0,9,2
1,10,1
2,11,2
3,12,4
4,13,3
5,14,5
6,15,5
7,16,2
8,17,2
9,18,7
